In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [2]:
from combine import recombine
data = recombine("../data/potholes_1.csv", "../data/potholes_2.csv")

In [3]:
data["creation_date"] = pd.to_datetime(data["CREATION DATE"])
data["completion_date"] = pd.to_datetime(data["COMPLETION DATE"])
data["creation_year"] = pd.DatetimeIndex(data["creation_date"]).year

# Convert strings to times
data["creation_date"] = pd.to_datetime(data["creation_date"], format = "%m/%d/%Y")
data["completion_date"] = pd.to_datetime(data["completion_date"], format = "%m/%d/%Y")

# Create completion time variable
data["completion_time"] = data["completion_date"] - data["creation_date"]


In [4]:
data["completion_time"] = data["completion_time"].apply(lambda x: x.days)

In [5]:
data.drop('CREATION DATE', axis=1, inplace=True)
data.drop('COMPLETION DATE', axis=1, inplace=True)
data.drop('STATUS', axis=1, inplace=True)
data.drop('SERVICE REQUEST NUMBER', axis=1, inplace=True)
data.drop('TYPE OF SERVICE REQUEST', axis=1, inplace=True)
data.drop('CURRENT ACTIVITY', axis=1, inplace=True)
data.drop('MOST RECENT ACTION', axis=1, inplace=True)
data.drop('STREET ADDRESS', axis=1, inplace=True)
data.drop('X COORDINATE', axis=1, inplace=True)
data.drop('Y COORDINATE', axis=1, inplace=True)
data.drop('SSA', axis=1, inplace=True)
data.drop('LATITUDE', axis=1, inplace=True)
data.drop('LONGITUDE', axis=1, inplace=True)
data.drop('LOCATION', axis=1, inplace=True)
data.drop('creation_date', axis=1, inplace=True)
data.drop('completion_date', axis=1, inplace=True)
data.drop('ZIP', axis=1, inplace=True)
data.drop('Ward', axis=1, inplace=True)
data.drop('Police District', axis=1, inplace=True)
data.drop('NUMBER OF POTHOLES FILLED ON BLOCK', axis=1, inplace=True)
#data.drop('creation_year', axis=1, inplace=True)
data = data.dropna()
data["community_area"] = data["Community Area"].astype(int)
data.drop('Community Area', axis=1, inplace=True)
data.head


<bound method NDFrame.head of         creation_year  completion_time  community_area
0                2014            405.0               0
4                2018              4.0              24
5                2014             78.0              30
11               2014             19.0               0
13               2016              3.0              24
...               ...              ...             ...
560473           2014              0.0               3
560474           2011              0.0              32
560475           2015              0.0               8
560476           2014             31.0              77
560477           2015             75.0              24

[552844 rows x 3 columns]>

In [14]:
#data1 = data.groupby(["community_area"])["creation_year"].count()
#data1 = data1.drop(labels=0, axis=0)

#Y1 = data1["creation_year"]
#Y1.head

data1 = data.groupby(["community_area"])["creation_year"].count().reset_index(name="count")
data1 = data1.drop(labels=0, axis=0)
data1.head
Y1 = data1["count"]
Y1.reset_index(drop=True)
Y1.head
#data2 = data.groupby(["community_area"]).agg({'completion_time': 'mean'})
#data2 = data2.drop(labels=0, axis=0)


<bound method NDFrame.head of 1      9960
2     15420
3      5415
4      7069
5      6335
      ...  
73     7627
74     5097
75     6693
76     2278
77     8928
Name: count, Length: 77, dtype: int64>

In [12]:
crime = pd.read_csv("../data/Crimes_data.csv")

crime["community_area"] = crime["Community Area"]
crimes = crime[['community_area', "ID"]]
crimes.dropna()

crimec = crimes.groupby(["community_area"]).count()
crimec.head


<bound method NDFrame.head of                   ID
community_area      
1               3503
2               2347
3               3474
4               1343
5                746
...              ...
73              3104
74               374
75              1994
76              2198
77              1961

[77 rows x 1 columns]>

In [8]:
income = pd.read_csv("../data/chicago_census_data.csv")

income = income.drop(labels=77, axis=0)
income["income"] = income["PER CAPITA INCOME "]
income["community_area"] = income["Community Area Number"].astype(int)
incomes = income[['community_area', "income"]]

incomes.head

<bound method NDFrame.head of     community_area  income
0                1   23939
1                2   23040
2                3   35787
3                4   37524
4                5   57123
..             ...     ...
72              73   19713
73              74   34381
74              75   27149
75              76   25828
76              77   33385

[77 rows x 2 columns]>

In [15]:
X = pd.merge(incomes, crimec, on = 'community_area')
X.drop('community_area', axis=1, inplace=True)
X.index += 1
X.head

<bound method NDFrame.head of     income    ID
1    23939  3503
2    23040  2347
3    35787  3474
4    37524  1343
5    57123   746
..     ...   ...
73   19713  3104
74   34381   374
75   27149  1994
76   25828  2198
77   33385  1961

[77 rows x 2 columns]>

In [19]:
X2 = sm.add_constant(X)
est = sm.OLS(Y1, X2)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  count   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     12.69
Date:                Mon, 21 Mar 2022   Prob (F-statistic):           1.83e-05
Time:                        01:46:49   Log-Likelihood:                -738.35
No. Observations:                  77   AIC:                             1483.
Df Residuals:                      74   BIC:                             1490.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3355.4005    923.551      3.633      0.001    1515.186    5195.615
income         0.0868      0.027      3.188      0.002       0.033       0.141
ID             0.4341      0.102      4.244      0.000       0.230       0.638
==============================================================================
Omnibus:                        4.793   Durbin-Watson:                   1.674
Prob(Omnibus):                  0.091   Jarque-Bera (JB):                4.681
Skew:                           0.600   Prob(JB):                       0.0963
Kurtosis:                       2.868   Cond. No.                     6.72e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.72e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""